In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load your dataframe here
# df = pd.read_csv('your_file.csv')
df=pd.read_csv("C:\\Users\\saipr\\Downloads\\Combined Data.csv (1)\\Combined Data.csv")

df=df.dropna(subset=['statement'])

 


In [9]:
df

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...,...
53038,53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety
53039,53039,"selfishness ""I don't feel very good, it's lik...",Anxiety
53040,53040,Is there any way to sleep better? I can't slee...,Anxiety
53041,53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety


In [15]:
df['status'].value_counts()

status
Normal                  16343
Depression              15404
Suicidal                10652
Anxiety                  3841
Bipolar                  2777
Stress                   2587
Personality disorder     1077
Name: count, dtype: int64

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

# Load your dataframe here
# df = pd.read_csv('your_file.csv')

# Extract features and labels
texts = df['statement'].values
labels = df['status'].values

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
one_hot_labels = to_categorical(encoded_labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, one_hot_labels, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=10000)  # Adjust the num_words parameter as needed
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform input length
max_sequence_length = 100  # Adjust as needed
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(units=64, return_sequences=False, kernel_regularizer=l2(0.01)))  # L2 regularization
model.add(Dropout(0.6))  # Increased dropout rate
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping and model checkpoint callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, save_weights_only=False)

# Train the model
history = model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

# Load the best model
model.load_weights('best_model.keras')

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


Epoch 1/10


c:\Users\saipr\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1054/1054 ━━━━━━━━━━━━━━━━━━━━ 64s 58ms/step - accuracy: 0.5376 - loss: 1.4708 - val_accuracy: 0.6312 - val_loss: 0.8663
Epoch 2/10
1054/1054 ━━━━━━━━━━━━━━━━━━━━ 43s 40ms/step - accuracy: 0.6410 - loss: 0.8543 - val_accuracy: 0.6842 - val_loss: 0.8026
Epoch 3/10
1054/1054 ━━━━━━━━━━━━━━━━━━━━ 44s 42ms/step - accuracy: 0.7234 - loss: 0.7253 - val_accuracy: 0.6994 - val_loss: 0.7772
Epoch 4/10
1054/1054 ━━━━━━━━━━━━━━━━━━━━ 45s 43ms/step - accuracy: 0.7581 - loss: 0.6607 - val_accuracy: 0.7229 - val_loss: 0.7632
Epoch 5/10
1054/1054 ━━━━━━━━━━━━━━━━━━━━ 46s 44ms/step - accuracy: 0.7915 - loss: 0.6026 - val_accuracy: 0.7182 - val_loss: 0.7943
Epoch 6/10
1054/1054 ━━━━━━━━━━━━━━━━━━━━ 45s 42ms/step - accuracy: 0.8057 - loss: 0.5577 - val_accuracy: 0.7299 - val_loss: 0.7380
Epoch 7/10
1054/1054 ━━━━━━━━━━━━━━━━━━━━ 45s 43ms/step - accuracy: 0.8190 - loss: 0.5283 - val_accuracy: 0.7369 - val_loss: 0.7410
Epoch 8/10
1054/1054 ━━━━━━━━━━━━━━━━━━━━ 47s 45ms/step - accuracy: 0.8343 - loss: 0.49

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Assuming you have the following variables already set up:
# tokenizer - the Tokenizer object used for training
# max_sequence_length - the maximum length of the sequences
# model - the trained model or a path to the saved model

# Load the saved model
model = load_model('best_model.keras')

# Example new texts for prediction
new_texts = [
    "Balancing work, family, and personal responsibilities is leaving me feeling drained and anxious all the time",
    "I have had a few dreams where something is trying to push me off a cliff, i always struggling trying to stay on for a bit but then just give up and jump off because id rather die than struggle to live. my day thoughts are taking over my night thoughts..."
]

# Tokenize and pad the new texts
X_new_sequences = tokenizer.texts_to_sequences(new_texts)
X_new_padded = pad_sequences(X_new_sequences, maxlen=max_sequence_length)

# Make predictions
predictions = model.predict(X_new_padded)

# Convert predictions to labels
# If using one-hot encoding, you may need to use np.argmax to get the predicted class index
predicted_class_indices = np.argmax(predictions, axis=1)

# Decode the predicted class indices to labels
# Assuming label_encoder is already fit on the original labels
predicted_labels = label_encoder.inverse_transform(predicted_class_indices)

# Print the predictions
for text, label in zip(new_texts, predicted_labels):
    print(f'Text: "{text}"\nPredicted Label: {label}\n')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
Text: "Balancing work, family, and personal responsibilities is leaving me feeling drained and anxious all the time"
Predicted Label: Anxiety

Text: "I have had a few dreams where something is trying to push me off a cliff, i always struggling trying to stay on for a bit but then just give up and jump off because id rather die than struggle to live. my day thoughts are taking over my night thoughts..."
Predicted Label: Suicidal



In [14]:
import joblib

# Save the model and vectorizer
joblib.dump(model, 'model.pkl')

['model.pkl']

In [16]:
# Save tokenizer and label encoder
joblib.dump(tokenizer, 'tokenizer.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')


['label_encoder.pkl']

In [19]:
model.save('best_model.keras')
